In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('/home/tammy.kojima/Authorship-attribution/')
import feature_extraction as fe
from tqdm import tqdm
import pickle
import os
import ast
from sklearn.model_selection import train_test_split

/home/tammy.kojima/miniconda3/envs/meu_ambiente/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-07 12:20:20,627	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-09-07 12:20:20,738	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
df_mistral = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_mistral.csv')
df_human = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_human.csv')
df_mistral = df_mistral.iloc[:5000]
df_human = df_human.iloc[:5000]
df_test = pd.concat([df_mistral, df_human], ignore_index=True)
df_test

,tweet_id,text,topic,length,batch_timestamp,characters_remaining,origin,model
0,1.0,"""The future is here: AI-powered virtual tutors...",artificial intelligence in education,193.0,2025-09-07 11:05:21,87.0,NaN,NaN
1,2.0,"""From predictive analytics that identify strug...",artificial intelligence in education,192.0,2025-09-07 11:05:21,88.0,NaN,NaN
2,3.0,"""Empower your classroom with AI-driven solutio...",artificial intelligence in education,166.0,2025-09-07 11:05:21,114.0,NaN,NaN
3,4.0,"""Let's debunk the myth - AI isn't replacing hu...",artificial intelligence in education,237.0,2025-09-07 11:05:21,43.0,NaN,NaN
4,5.0,Ever wondered how AI can make teachers' lives ...,artificial intelligence in education,188.0,2025-09-07 11:05:21,92.0,NaN,NaN
...,...,...,...,...,...,...,...,...
9995,NaN,i've gone to 3rd place! http://bit.ly/VwV6H p...,NaN,NaN,NaN,NaN,human,NaN
9996,NaN,@freshome I read a book that tells exacly that...,NaN,NaN,NaN,NaN,human,NaN
9997,NaN,@ConnecticutSun its ok pic is kind of fuzzy bu...,NaN,NaN,NaN,NaN,human,NaN
9998,NaN,Feel Like working over the weekend.. I hope i ...,NaN,NaN,NaN,NaN,human,NaN


In [3]:
# Tokeniza todos os textos primeiro
textos = df_test["text"].fillna("").tolist()
processed_texts = [fe.preprocess_text(texto) for texto in tqdm(textos, desc="Tokenizando textos")]

Tokenizando textos:   0%|                                                                                                                        | 0/10000 [00:00<?, ?it/s][nltk_data] Error loading punkt: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
Tokenizando textos: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 14653.97it/s]


In [6]:
df_prossessed_texts = pd.DataFrame({'text': processed_texts})
df_prossessed_texts.to_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_tokens_mistral.csv', index=False)

In [7]:
# Lista para armazenar resultados
lexical_features = []

for texto in tqdm(textos):
    # Tokeniza o texto usando Twokenizer
    words = fe.preprocess_text(texto)  # já retorna tokens em lowercase
    
    # Extrai features léxicas
    features = fe.extract_lexical_features(texto, words)
    lexical_features.append(features)

df_lexical = pd.DataFrame(lexical_features)
df_test = pd.concat([df_test, df_lexical], axis=1)
df_test.head()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 15850.27it/s]


,tweet_id,text,topic,length,batch_timestamp,characters_remaining,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio
0,1.0,"""The future is here: AI-powered virtual tutors...",artificial intelligence in education,193.0,2025-09-07 11:05:21,87.0,NaN,NaN,0.933333,30,28,5.666667,18.288889,0.100000
1,2.0,"""From predictive analytics that identify strug...",artificial intelligence in education,192.0,2025-09-07 11:05:21,88.0,NaN,NaN,0.935484,31,29,5.387097,16.237253,0.129032
2,3.0,"""Empower your classroom with AI-driven solutio...",artificial intelligence in education,166.0,2025-09-07 11:05:21,114.0,NaN,NaN,0.896552,29,26,5.137931,14.325803,0.068966
3,4.0,"""Let's debunk the myth - AI isn't replacing hu...",artificial intelligence in education,237.0,2025-09-07 11:05:21,43.0,NaN,NaN,0.933333,45,42,4.511111,11.227654,0.088889
4,5.0,Ever wondered how AI can make teachers' lives ...,artificial intelligence in education,188.0,2025-09-07 11:05:21,92.0,NaN,NaN,1.000000,34,34,4.735294,10.429931,0.088235


In [8]:
stylistic_features = [
    fe.extract_stylistic_features(textos[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estilísticas")
]
df_stylistic = pd.DataFrame(stylistic_features)
df_test = pd.concat([df_test, df_stylistic], axis=1)
df_test.head()

Extraindo features estilísticas: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 109506.71it/s]


,tweet_id,text,topic,length,batch_timestamp,characters_remaining,origin,model,lexical_type_token_ratio,lexical_word_count,...,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,stylistic_question_density,stylistic_ellipsis_count,stylistic_emoji_density,stylistic_emoticon_density
0,1.0,"""The future is here: AI-powered virtual tutors...",artificial intelligence in education,193.0,2025-09-07 11:05:21,87.0,NaN,NaN,0.933333,30,...,5.666667,18.288889,0.100000,0,0,0.066667,0.000000,0,0.000000,0.0
1,2.0,"""From predictive analytics that identify strug...",artificial intelligence in education,192.0,2025-09-07 11:05:21,88.0,NaN,NaN,0.935484,31,...,5.387097,16.237253,0.129032,0,0,0.032258,0.000000,0,0.032258,0.0
2,3.0,"""Empower your classroom with AI-driven solutio...",artificial intelligence in education,166.0,2025-09-07 11:05:21,114.0,NaN,NaN,0.896552,29,...,5.137931,14.325803,0.068966,1,0,0.068966,0.000000,1,0.068966,0.0
3,4.0,"""Let's debunk the myth - AI isn't replacing hu...",artificial intelligence in education,237.0,2025-09-07 11:05:21,43.0,NaN,NaN,0.933333,45,...,4.511111,11.227654,0.088889,0,0,0.022222,0.000000,0,0.044444,0.0
4,5.0,Ever wondered how AI can make teachers' lives ...,artificial intelligence in education,188.0,2025-09-07 11:05:21,92.0,NaN,NaN,1.000000,34,...,4.735294,10.429931,0.088235,0,0,0.029412,0.029412,0,0.029412,0.0


In [9]:
structural_features = [
    fe.extract_structural_features(textos[i], processed_texts[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estruturais")
]
df_structural = pd.DataFrame(structural_features)
df_test = pd.concat([df_test, df_structural], axis=1)
df_test.head()

Extraindo features estruturais: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 136580.91it/s]


,tweet_id,text,topic,length,batch_timestamp,characters_remaining,origin,model,lexical_type_token_ratio,lexical_word_count,...,stylistic_emoticon_density,structural_has_url,structural_has_mention,structural_has_hashtag,structural_is_retweet,structural_url_density,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers
0,1.0,"""The future is here: AI-powered virtual tutors...",artificial intelligence in education,193.0,2025-09-07 11:05:21,87.0,NaN,NaN,0.933333,30,...,0.0,0,0,1,0,0.0,0.0,0.033333,0.0,0.0
1,2.0,"""From predictive analytics that identify strug...",artificial intelligence in education,192.0,2025-09-07 11:05:21,88.0,NaN,NaN,0.935484,31,...,0.0,0,0,1,0,0.0,0.0,0.032258,0.0,0.0
2,3.0,"""Empower your classroom with AI-driven solutio...",artificial intelligence in education,166.0,2025-09-07 11:05:21,114.0,NaN,NaN,0.896552,29,...,0.0,0,0,0,0,0.0,0.0,0.000000,0.0,0.0
3,4.0,"""Let's debunk the myth - AI isn't replacing hu...",artificial intelligence in education,237.0,2025-09-07 11:05:21,43.0,NaN,NaN,0.933333,45,...,0.0,0,0,1,0,0.0,0.0,0.022222,0.0,0.0
4,5.0,Ever wondered how AI can make teachers' lives ...,artificial intelligence in education,188.0,2025-09-07 11:05:21,92.0,NaN,NaN,1.000000,34,...,0.0,0,0,1,0,0.0,0.0,0.029412,0.0,0.0


In [10]:
syntactic_features = [
    fe.extract_syntactic_features(tokens)
    for tokens in tqdm(processed_texts, desc="Extraindo features sintáticas")
]
df_syntactic = pd.DataFrame(syntactic_features)
df_test = pd.concat([df_test, df_syntactic], axis=1)
df_test.head()

Extraindo features sintáticas: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:47<00:00, 209.24it/s]


,tweet_id,text,topic,length,batch_timestamp,characters_remaining,origin,model,lexical_type_token_ratio,lexical_word_count,...,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers,syntactic_pos_tag_entropy,syntactic_pos_bigram_entropy,syntactic_avg_sentence_length,syntactic_subordinating_conj,syntactic_comma_ratio,syntactic_punct_ratio
0,1.0,"""The future is here: AI-powered virtual tutors...",artificial intelligence in education,193.0,2025-09-07 11:05:21,87.0,NaN,NaN,0.933333,30,...,0.0,0.033333,0.0,0.0,2.186646,3.272991,7.000000,0.0,0.000000,0.433333
1,2.0,"""From predictive analytics that identify strug...",artificial intelligence in education,192.0,2025-09-07 11:05:21,88.0,NaN,NaN,0.935484,31,...,0.0,0.032258,0.0,0.0,2.133447,3.232776,14.000000,0.0,0.032258,0.161290
2,3.0,"""Empower your classroom with AI-driven solutio...",artificial intelligence in education,166.0,2025-09-07 11:05:21,114.0,NaN,NaN,0.896552,29,...,0.0,0.000000,0.0,0.0,1.916567,2.999816,5.750000,0.0,0.068966,0.344828
3,4.0,"""Let's debunk the myth - AI isn't replacing hu...",artificial intelligence in education,237.0,2025-09-07 11:05:21,43.0,NaN,NaN,0.933333,45,...,0.0,0.022222,0.0,0.0,2.235856,3.322242,13.333333,0.0,0.044444,0.222222
4,5.0,Ever wondered how AI can make teachers' lives ...,artificial intelligence in education,188.0,2025-09-07 11:05:21,92.0,NaN,NaN,1.000000,34,...,0.0,0.029412,0.0,0.0,2.251875,3.376443,7.250000,0.0,0.029412,0.235294


In [11]:
ngrams_df = fe.extract_ngrams_features(processed_texts, max_features=300)
ngrams_df.head()

,about,accessible,adaptive,ai,ai can,ai in,ai in education,ai is,ai is revolutionizing,ai powered,...,where,will,with,with ai,work,yes,you,you know,you ready,your
0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [26]:
# Resetar os índices de ambos os DataFrames
df_test_reset = df_test.reset_index(drop=True)
ngrams_df_reset = ngrams_df.reset_index(drop=True)

# Agora concatenar
df_combined = pd.concat([df_test_reset, ngrams_df_reset], axis=1)
df_combined['origin'] = df_combined['origin'].replace({'AI': 1, 'human': 0})
df_combined.fillna(0)
df_combined.head()

/tmp/ipykernel_421706/3577768971.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined['origin'] = df_combined['origin'].replace({'AI': 1, 'human': 0})


,tweet_id,text,topic,length,batch_timestamp,characters_remaining,origin,model,lexical_type_token_ratio,lexical_word_count,...,where,will,with,with ai,work,yes,you,you know,you ready,your
0,1.0,"""The future is here: AI-powered virtual tutors...",artificial intelligence in education,193.0,2025-09-07 11:05:21,87.0,NaN,NaN,0.933333,30,...,0,0,0,0,0,0,0,0,0,0
1,2.0,"""From predictive analytics that identify strug...",artificial intelligence in education,192.0,2025-09-07 11:05:21,88.0,NaN,NaN,0.935484,31,...,0,0,0,0,0,0,0,0,0,0
2,3.0,"""Empower your classroom with AI-driven solutio...",artificial intelligence in education,166.0,2025-09-07 11:05:21,114.0,NaN,NaN,0.896552,29,...,0,0,1,1,0,0,0,0,0,1
3,4.0,"""Let's debunk the myth - AI isn't replacing hu...",artificial intelligence in education,237.0,2025-09-07 11:05:21,43.0,NaN,NaN,0.933333,45,...,0,0,0,0,0,0,0,0,0,0
4,5.0,Ever wondered how AI can make teachers' lives ...,artificial intelligence in education,188.0,2025-09-07 11:05:21,92.0,NaN,NaN,1.000000,34,...,0,0,1,0,0,0,0,0,0,0


In [27]:
df_combined = df_combined.drop(columns=['topic', 'model', 'tweet_id', 'text', 'length', 'batch_timestamp', 'characters_remaining']) 
df_combined.head()

,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,...,where,will,with,with ai,work,yes,you,you know,you ready,your
0,NaN,0.933333,30,28,5.666667,18.288889,0.100000,0,0,0.066667,...,0,0,0,0,0,0,0,0,0,0
1,NaN,0.935484,31,29,5.387097,16.237253,0.129032,0,0,0.032258,...,0,0,0,0,0,0,0,0,0,0
2,NaN,0.896552,29,26,5.137931,14.325803,0.068966,1,0,0.068966,...,0,0,1,1,0,0,0,0,0,1
3,NaN,0.933333,45,42,4.511111,11.227654,0.088889,0,0,0.022222,...,0,0,0,0,0,0,0,0,0,0
4,NaN,1.000000,34,34,4.735294,10.429931,0.088235,0,0,0.029412,...,0,0,1,0,0,0,0,0,0,0


In [28]:
df_combined['origin'] = df_combined['origin'].fillna(1)
df_combined.head()

,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,...,where,will,with,with ai,work,yes,you,you know,you ready,your
0,1.0,0.933333,30,28,5.666667,18.288889,0.100000,0,0,0.066667,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0.935484,31,29,5.387097,16.237253,0.129032,0,0,0.032258,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0.896552,29,26,5.137931,14.325803,0.068966,1,0,0.068966,...,0,0,1,1,0,0,0,0,0,1
3,1.0,0.933333,45,42,4.511111,11.227654,0.088889,0,0,0.022222,...,0,0,0,0,0,0,0,0,0,0
4,1.0,1.000000,34,34,4.735294,10.429931,0.088235,0,0,0.029412,...,0,0,1,0,0,0,0,0,0,0


In [29]:
df_combined.to_csv("/home/tammy.kojima/Authorship-attribution/df_pronto/df_mistral_com_features.csv", index=False)

In [5]:
df_combined = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_mistral_com_features.csv')
X_new_features = fe.extract_features_batch(textos, processed_texts = processed_texts)
df_features = pd.DataFrame(X_new_features)
df_combined = pd.concat([df_combined, df_features], axis=1)
df_combined.head()

,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,...,chatgpt_structured_output,chatgpt_overly_polite,chatgpt_disclaimer_density,chatgpt_assistant_patterns,mistral_self_ref,mistral_structured_density,mistral_technical_jargon,mistral_non_english_density,mistral_step_reasoning,mistral_low_ethical_disclaimers
0,1.0,0.933333,30,28,5.666667,18.288889,0.100000,0,0,0.066667,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,1.0,0.935484,31,29,5.387097,16.237253,0.129032,0,0,0.032258,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,1.0,0.896552,29,26,5.137931,14.325803,0.068966,1,0,0.068966,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,1.0,0.933333,45,42,4.511111,11.227654,0.088889,0,0,0.022222,...,0.022222,0.0,0.0,0.0,0.0,0.022222,0.0,0.0,0.0,0.0
4,1.0,1.000000,34,34,4.735294,10.429931,0.088235,0,0,0.029412,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [6]:
df_combined.to_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_mistral_com_features2.csv', index=False)